# 构建基本聊天机器人

### 安装所需的软件包并配置您的环境

In [24]:
# %%capture --no-stderr
# %pip install -U langgraph langsmith langchain_anthropic

首先创建一个StateGraph。StateGraph对象将我们的聊天机器人的结构定义为“状态机”。我们将添加nodes以表示我们的聊天机器人可以调用的 llm 和函数，并edges指定机器人应如何在这些函数之间转换。

我们的图表现在可以处理两个关键任务：

1、每个都node可以接收当前State作为输入并输出状态更新。

2、messages由于语法add_messages中使用了预建函数，更新将被附加到现有列表而不是覆盖它。Annotated

In [25]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

接下来，添加一个“ chatbot”节点。节点代表工作单元。它们通常是常规的 Python 函数。

**注意**
节点函数如何chatbot将当前State作为输入并返回一个包含messages键“messages”下更新列表的字典。这是所有 LangGraph 节点函数的基本模式。

add_messages我们的函数会将Statellm 的响应消息附加到状态中已有的任何消息中。

In [26]:
from typing import Annotated, Any
from typing_extensions import TypedDict
from langchain.schema.messages import HumanMessage
import requests
import os



# 设置 SiliconFlow API 的 URL 和密钥
SILICONFLOW_API_URL = "https://api.siliconflow.cn/v1/chat/completions"
API_KEY = os.getenv("SILICONFLOW_API_KEY", "sk-iihecohtojjkbgwaqaoukdopasitiyneinjstosyadytfepl")
print(API_KEY)
def convert_message(message: Any) -> dict:
    """将 LangChain 消息对象转换为普通字典"""
    if isinstance(message, HumanMessage):
        return {"role": "user", "content": message.content}
    # 如果有其他消息类型，可以在这里添加转换逻辑
    return message

def chatbot(state: State):
    # 从 State 中提取用户消息
    messages = state["messages"]
    # 将消息转换为普通字典
    converted_messages = [convert_message(msg) for msg in messages]
    
    # 构建请求 payload
    payload = {
        "model": "Qwen/Qwen2.5-7B-Instruct",
        "messages": converted_messages,
        "stream": False,
        "max_tokens": 512,
        "temperature": 0.7,
        "top_p": 0.7,
        "top_k": 50,
        "frequency_penalty": 0.5,
        "response_format": {"type": "text"}
    }
    
    # 设置请求头
    headers = {
        "Authorization": f"Bearer {API_KEY}", #API
        "Content-Type": "application/json"
    }
    
    # 发送请求
    try:
        response = requests.post(SILICONFLOW_API_URL, json=payload, headers=headers)
        response.raise_for_status()  # 检查请求是否成功
        response_data = response.json()
        
        # 提取响应中的消息内容
        if "choices" in response_data and len(response_data["choices"]) > 0:
            assistant_message = response_data["choices"][0]["message"]["content"]
            return {"messages": [{"role": "assistant", "content": assistant_message}]}
        else:
            return {"messages": [{"role": "assistant", "content": "No response from SiliconFlow API"}]}
    
    except Exception as e:
        return {"messages": [{"role": "assistant", "content": f"Error: {str(e)}"}]}

# 添加 chatbot 节点
graph_builder.add_node("chatbot", chatbot)

sk-iihecohtojjkbgwaqaoukdopasitiyneinjstosyadytfepl


接下来，添加一个entry点。这将告诉我们的图表每次运行时**从哪里开始工作。**

In [27]:
graph_builder.add_edge(START, "chatbot")

同样，设置一个finish点。这将指示图表“任何时候运行此节点，您都可以退出。”

In [28]:
graph_builder.add_edge("chatbot", END)

最后，我们希望能够运行我们的图表。为此，请compile()在图表构建器上调用“”。这将创建一个“ CompiledGraph”，我们可以在状态上使用“调用”。

In [29]:
graph = graph_builder.compile()

get_graph您可以使用方法和其中一种“绘制”方法（例如draw_ascii或）来可视化图形draw_png。draw每个方法都需要额外的依赖项。

In [30]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

现在让我们运行聊天机器人！

**提示：**您可以随时通过输入“quit”、“exit”或“q”退出聊天循环。

In [ ]:
# def stream_graph_updates(user_input: str):
#     for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
#         for value in event.values():
#             print("Assistant:", value["messages"][-1].content)

def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        # print("Event:", event)  # 打印事件内容
        for value in event.values():
            print("Value:", value)  # 打印值
            if "messages" in value and len(value["messages"]) > 0:
                last_message = value["messages"][-1]
                if hasattr(last_message, "content"):
                    print("Assistant:", last_message.content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

Value: {'messages': [{'role': 'assistant', 'content': '我是Qwen，由阿里云开发的人工智能模型。我被设计用来提供信息查询、语言翻译、创意写作等多种语言服务。很高兴能为你提供帮助！'}]}
Value: {'messages': [{'role': 'assistant', 'content': '当然可以！我可以帮助你进行以下任务：\n\n1. **信息查询**：提供各种领域的知识，包括但不限于科技、文化、历史、地理等。\n2. **语言翻译**：帮助你将文本从一种语言翻译成另一种语言。\n3. **写作辅助**：提供文章构思、润色修改、创作故事或剧本等帮助。\n4. **学习辅导**：解答学习中遇到的问题，提供学习资料推荐等。\n5. **日常生活建议**：提供旅行建议、健康饮食建议、生活小技巧等。\n6. **编程和代码帮助**：提供编程问题解答，代码调试等帮助。\n7. **心理支持**：提供心理咨询服务，倾听你的困扰并给出建议。\n\n请告诉我你需要什么帮助，我会尽力为你提供支持。'}]}
Goodbye!
